In [ ]:
# 백본 모델 : MovileNetV2
# 희귀한 소량의 이미지 데이터는 cifar10 데이터로 대신함
# 실습2 : movilenetv2 모델 그대로 학습시켜 내 이미지 데이터 분류하는 모델 생성
# Transfer learning, Fine-tunning

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
(x_train, y_train), (x_test,y_test) = keras.datasets.cifar10.load_data()
# CIFAR-10은 10개의 클래스로 이루어진 32x32 크기의 이미지 데이터셋
print(x_train.shape)
num_classes = 10

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 14s 0us/step
(50000, 32, 32, 3)


In [ ]:
# 픽셀 값을 0.0 ~ 1.0 범위로 정규화
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

# 레이블을 원-핫 인코딩(one-hot encoding)으로 변환
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
#  to_categorical은 다중 클래스 분류 문제에서 레이블을 신경망이 이해하기 쉬운 형태로 변환하는 필수적인 전처리 과정. categorical_crossentropy 손실 함수를 사용하려면 반드시 레이블을 원-핫 인코딩해야 함.

In [ ]:
# 전이학습 (Transfer learning) : 기본 모델의 가중치는 모두 동결(freeze) - > 새로 추가한 분류층만 학습


# MobileNetV2 모델을 ImageNet 가중치와 함께 로드.
# input_shape=(96,96,3): 모델이 96x96x3 크기의 이미지를 입력받도록 설정.
# include_top=False: 기존 모델의 최종 분류 레이어를 제외하고, 특징 추출 부분만 가져옴.

base_model = keras.applications.MobileNetV2(
    input_shape = (96,96,3),
    include_top = False,  # 분류기 부분을 빼고 컨볼루션만 해당
    weights = "imagenet"
)

# base_model의 가중치를 동결(freeze).
# 이로 인해 이 레이어들은 학습 과정에서 가중치가 업데이트되지 않음.
base_model.trainable = False

# 함수형 api로 모델생성

inputs = keras.Input(shape=(32,32,3))
# 입력 이미지를 base_model이 요구하는 크기(96x96)로 리사이즈(resize)
x = layers.Resizing(96,96)(inputs)
# 동결된 base_model을 통과시켜 이미지 특징을 추출
x = base_model(x, training=False)
# GlobalAveragePooling2D: 추출된 특징 맵의 평균을 내어 1차원 벡터로 만듦.
# 이는 파라미터 수를 줄여 과적합을 방지.
x = layers.GlobalAveragePooling2D()(x) # Maxpooling 보다 더 급격하게 feature의 크기를 줄임

# num_classes: CIFAR-10의 10개 클래스를 분류하기 위한 Dense 레이어를 추가.
# activation="softmax": 각 클래스에 대한 확률을 출력.
outputs = layers.Dense(num_classes, activation="softmax")(x)
# 입력과 출력을 연결하여 새로운 모델을 정의
model_t1 = keras.Model(inputs, outputs)
model_t1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 모델을 훈련. 이 단계에서는 새로 추가한 Dense 레이어만 학습
model_t1.fit(x_train, y_train, epochs=5, validation_split=0.1, batch_size=64, verbose=1)

print('test평가결과 :', model_t1.evaluate(x_test, y_test))


# keras.applications.MobileNetV2와 같은 사전 학습 모델은 **백본(Backbone)**이라고 불림.
# 이 백본은 이미 강력한 특징 추출 능력을 갖추고 있어, 적은 데이터로도 빠르게 좋은 성능을 낼 수 있음.
# training=False는 base_model이 추론 모드(inference mode)로 작동하도록 하여 학습 중에 가중치가 변경되지 않도록 함.

Epoch 1/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 40s 36ms/step - accuracy: 0.6517 - loss: 1.0129 - val_accuracy: 0.7714 - val_loss: 0.6428
Epoch 2/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 9s 13ms/step - accuracy: 0.7950 - loss: 0.5961 - val_accuracy: 0.7870 - val_loss: 0.5982
Epoch 3/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 9s 13ms/step - accuracy: 0.8114 - loss: 0.5453 - val_accuracy: 0.8126 - val_loss: 0.5575
Epoch 4/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 9s 13ms/step - accuracy: 0.8236 - loss: 0.5056 - val_accuracy: 0.8010 - val_loss: 0.5689
Epoch 5/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - accuracy: 0.8250 - loss: 0.4992 - val_accuracy: 0.8066 - val_loss: 0.5608
313/313 ━━━━━━━━━━━━━━━━━━━━ 21s 37ms/step - accuracy: 0.8015 - loss: 0.5778
test평가결과 : [0.5780776739120483, 0.7996000051498413]


In [ ]:

# Fine-Tunning(미세조정)
# 베이스 모델 일부 층만 열기 (예: 마지막 30개)

# 베이스 모델의 동결을 해제. 이제 전체 모델이 학습될 수 있는 상태가 됨.
base_model.trainable = True
# base_model의 모든 레이어 중 마지막 30개만 학습 가능하도록 설정
# 이는 이미지의 '고수준 특징'을 추출하는 레이어만 재학습하기위해
for layer in  base_model.layers[:-30]:
    layer.trainable = False

# 이제 낮은 학습률(learning-rate)로 재컴파일
# weight를 너무 변화를 주면 안됨
model_t1 = keras.Model(inputs, outputs)
model_t1.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
model_t1.fit(x_train, y_train, epochs=5, validation_split=0.1, batch_size=64, verbose=1)

print('test평가결과 :', model_t1.evaluate(x_test, y_test))

# 미세 조정은 전이 학습보다 더 나은 결과를 얻기 위한 중요한 단계.
# 낮은 학습률은 이미 최적화된 가중치에 미세한 조정을 가하여, 새로운 데이터셋의 고유한 특징에 더욱 정밀하게 맞추는 역할.
# 이는 모델의 성능을 극한으로 끌어올리는 효과적인 방법.

Epoch 1/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 40s 36ms/step - accuracy: 0.6877 - loss: 1.0658 - val_accuracy: 0.7958 - val_loss: 0.6616
Epoch 2/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 12s 17ms/step - accuracy: 0.8003 - loss: 0.6233 - val_accuracy: 0.8094 - val_loss: 0.6072
Epoch 3/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 12s 17ms/step - accuracy: 0.8313 - loss: 0.5053 - val_accuracy: 0.8198 - val_loss: 0.5738
Epoch 4/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 12s 17ms/step - accuracy: 0.8545 - loss: 0.4200 - val_accuracy: 0.8218 - val_loss: 0.5669
Epoch 5/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 12s 17ms/step - accuracy: 0.8736 - loss: 0.3649 - val_accuracy: 0.8294 - val_loss: 0.5478
313/313 ━━━━━━━━━━━━━━━━━━━━ 8s 19ms/step - accuracy: 0.8267 - loss: 0.5556
test평가결과 : [0.5659272074699402, 0.8234999775886536]
